<a href="https://colab.research.google.com/github/davidvlaminck/AWVGeneric/blob/master/UseCases/AddKenmerkOnAssettype/AddKenmerkOnAssettype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pas eerst onderstaande parameter aan en voer het script uit. Een cookie vervalt binnen de 24 uur. Je kan [hier](https://sites.google.com/mow.vlaanderen.be/em-infra/afspraken-conventies/awv-cookie-ophalen) lezen hoe je de AWV ACM cookie kan ophalen.

In [1]:
awv_acm_cookie = 'f8571f44081c4260bdb5b23dd33b0b21'

Voer onderstaande script uit om de GitHub repo te clonen.

In [2]:
import os, shutil
if os.path.exists('AWVGeneric'):
  shutil.rmtree('AWVGeneric')
! git clone https://github.com/davidvlaminck/AWVGeneric

Cloning into 'AWVGeneric'...
remote: Enumerating objects: 862, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 862 (delta 233), reused 221 (delta 159), pack-reused 533 (from 1)
Receiving objects: 100% (862/862), 961.56 KiB | 5.49 MiB/s, done.
Resolving deltas: 100% (518/518), done.


Voer onderstaande script uit om kenmerken aan assettypes toe te voegen. Je kan de lijsten aanpassen. Vind de URI's van de assettypes op https://apps.mow.vlaanderen.be/eminfra/admin/assettypes. Je mag het script meermaals uitvoeren. Als het assettype het kenmerk reeds heeft, wordt dit als feedback geprint.

In [7]:
import sys
sys.path.insert(0, '/content/AWVGeneric')  # adds the Python files from the GitHub repo

import json
from pathlib import Path

from API.EMInfraClient import EMInfraClient
from API.Enums import Environment, AuthType

import pandas

path_to_excel_file = Path('Camera_relaties.xlsx')
name_relaties_tab = 'kenmerktypes'

eminfra_client = EMInfraClient(env=Environment.PRD, auth_type=AuthType.COOKIE, cookie=awv_acm_cookie)

relaties_tab = pandas.read_excel(path_to_excel_file, sheet_name=name_relaties_tab)
relaties_tab = relaties_tab[relaties_tab['aanwezig'] == 'nee'][['assettype', 'kenmerktype']]

assettypes = list(eminfra_client.get_all_assettypes())

# loop over all records in relaties_tab, using columns assettype and kenmerktype
for _, row in relaties_tab.iterrows():
    assettype_uri = row['assettype']
    kenmerktype_naam = row['kenmerktype']

    assettype = next((x for x in assettypes if x.uri == assettype_uri), None)
    if assettype is None:
        print(f'Assettype {assettype_uri} niet gevonden')
        continue

    kenmerktype = eminfra_client.get_kenmerktype_by_naam(kenmerktype_naam)
    if kenmerktype is None:
        print(f'Kenmerktype {kenmerktype_naam} niet gevonden')
        continue

    bestaande_kenmerken = eminfra_client.get_kenmerken_by_assettype_uuid(assettype.uuid)
    if any(x.kenmerkType.uuid == kenmerktype.uuid for x in bestaande_kenmerken):
        print(f'Kenmerk {kenmerktype.naam} reeds aanwezig op {assettype.uri}')
        continue

    eminfra_client.add_kenmerk_to_assettype(assettype_uuid=assettype.uuid,
                                            kenmerktype_uuid=kenmerktype.uuid)
    print(f'Kenmerk {kenmerktype.naam} toegevoegd aan {assettype.uri}')



                                            assettype      kenmerktype
0   https://lgc.data.wegenenverkeer.be/ns/installa...  Gemigreerd naar
1   https://lgc.data.wegenenverkeer.be/ns/installa...  Gemigreerd naar
2   https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
4   https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
6   https://lgc.data.wegenenverkeer.be/ns/installa...  Gemigreerd naar
7   https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
28  https://lgc.data.wegenenverkeer.be/ns/installa...  Gemigreerd naar
29  https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
31  https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
33  https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
37  https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
39  https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
41  https://lgc.data.wegenenverkeer.be/ns/installa...    Bevestigd aan
48  ht